# Protótipo de Detecção de Defeitos em Pacotes de Bolachas

In [ ]:

# Importando as bibliotecas necessárias
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import os
    

## 1. Configuração de Dados e Preprocessamento

In [ ]:

# Configuração de diretórios
base_dir = 'dados'  # Substituir pelo caminho onde estão as imagens
classes = ['perfeito', 'defeituoso']

# Leitura e preprocessamento de imagens
def carregar_imagens(base_dir, classes, img_size=(128, 128)):
    imagens = []
    labels = []
    for idx, classe in enumerate(classes):
        dir_classe = os.path.join(base_dir, classe)
        for arquivo in os.listdir(dir_classe):
            caminho_arquivo = os.path.join(dir_classe, arquivo)
            imagem = cv2.imread(caminho_arquivo)
            imagem = cv2.resize(imagem, img_size)
            imagens.append(imagem)
            labels.append(idx)
    imagens = np.array(imagens) / 255.0  # Normalização
    labels = to_categorical(labels, num_classes=len(classes))  # One-hot encoding
    return imagens, labels

# Carregando os dados
imagens, labels = carregar_imagens(base_dir, classes)
X_train, X_test, y_train, y_test = train_test_split(imagens, labels, test_size=0.2, random_state=42)
    

## 2. Construção do Modelo CNN

In [ ]:

# Construção do modelo
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(classes), activation='softmax')  # Saída com o número de classes
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
    

## 3. Augmentação de Dados e Treinamento

In [ ]:

# Augmentação de dados
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Treinamento
batch_size = 16
train_generator = datagen.flow(X_train, y_train, batch_size=batch_size)
history = model.fit(train_generator, epochs=10, validation_data=(X_test, y_test))
    

## 4. Teste em Tempo Real com Webcam

In [ ]:

# Captura e classificação em tempo real
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    # Pré-processamento do frame
    frame_resized = cv2.resize(frame, (128, 128))
    frame_normalized = frame_resized / 255.0
    prediction = model.predict(np.expand_dims(frame_normalized, axis=0))
    class_name = classes[np.argmax(prediction)]

    # Exibição do resultado
    cv2.putText(frame, class_name, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('Detecção de Defeitos', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
    